In [3]:
!pip install datasets

  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.12.0-py3-none-any.whl (183 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" 

In [4]:
from datasets import load_dataset
dataset = load_dataset("brunokreiner/genius-lyrics")

print(dataset)
 # Prints the dataset structure (DatasetDict)
print(dataset["train"].column_names) # Prints column names for the training split
print(dataset["train"].features) # Prints feature definitions

README.md:   0%|          | 0.00/2.63k [00:00<?, ?B/s]

english_lyrics_some_with_genres.csv:   0%|          | 0.00/663M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/480855 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'id', 'lyrics', 'is_english', 'genres_list', 'popularity', 'release_date', 'artist_id', 'artist_name', 'artist_popularity', 'artist_followers', 'artist_picture_url'],
        num_rows: 480855
    })
})
['Unnamed: 0', 'id', 'lyrics', 'is_english', 'genres_list', 'popularity', 'release_date', 'artist_id', 'artist_name', 'artist_popularity', 'artist_followers', 'artist_picture_url']
{'Unnamed: 0': Value(dtype='int64', id=None), 'id': Value(dtype='string', id=None), 'lyrics': Value(dtype='string', id=None), 'is_english': Value(dtype='bool', id=None), 'genres_list': Value(dtype='string', id=None), 'popularity': Value(dtype='float64', id=None), 'release_date': Value(dtype='string', id=None), 'artist_id': Value(dtype='string', id=None), 'artist_name': Value(dtype='string', id=None), 'artist_popularity': Value(dtype='float64', id=None), 'artist_followers': Value(dtype='float64', id=None), 'artist_picture_url': Value(dtype='str

In [5]:
import pandas as pd
dataset["train"].set_format(type="pandas")
train_df = dataset["train"][:]
print(train_df.head()) # Prints the first few rows as a DataFrame

lyrics_l = dataset["train"]["lyrics"].loc[dataset["train"]["is_english"] == True]
print(lyrics_l[:10])
print(len(lyrics_l))

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


   Unnamed: 0                      id  \
0           0  5FDh0G5oQ2UD9JxWiDCT1x   
1           1  6XK6Zw6JkFsHXzAcMWNiIr   
2           2  7GX5flRQZVHRAGd6B4TmDO   
3           3  73q3FpQVXWk5eSUnyo83E4   
4           4  1IdTIzkTK6CcpKKgMUGbg7   

                                              lyrics  is_english  \
0  paper doll come try it on step out of that bla...        True   
1  on my way on my way on my way on my way i woul...        True   
2  are you alright i m alright i m quite alright ...        True   
3  it s tearin up my heart when i m with you but ...        True   
4  some of us we re hardly ever here the rest of ...        True   

                                    genres_list  popularity release_date  \
0                                          None         NaN         None   
1                                          None         NaN         None   
2  ['melodic rap', 'philly rap', 'rap', 'trap']        82.0   2017-08-25   
3                                       

In [10]:
file_path = "lyrics.txt"

with open(file_path, "w") as file:
    for item in lyrics_l[:80000]:
        file.write(str(item) + "\n")

print(f"List saved to {file_path}")

List saved to lyrics.txt


In [7]:
!pip install transformers datasets evaluate peft accelerate bitsandbytes

  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Using cached bitsandbytes-0.45.5-py3-none-manylinux_2_24_x86_64.whl.metadata (5.0 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
Using cached evaluate-0.4.3-py3-none-any.whl (84 kB)
Using cached bitsandbytes-0.45.5-py3-none-manylinux_2_24_x86_64.whl (76.1 MB)
Using cached nvidia_cublas_cu12-12.4.5.8-p

In [6]:
!pip install -U bitsandbytes

In [7]:
!pip show transformers

Name: transformers
Version: 4.51.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: kaggle-environments, peft, sentence-transformers


In [12]:
# Lyrics Generator - Multi-GPU Version for 2x T4 GPUs

## Setup and Imports

# Installation of required packages
# Uncomment if needed
# !pip install transformers datasets evaluate peft accelerate bitsandbytes

import os
import math
import torch
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    IntervalStrategy # Although we use strings for strategy, keep import for reference
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType,
    PeftModel
)
import evaluate
import math
from IPython.display import display, Markdown, HTML
from transformers.trainer_callback import TrainerCallback # Import TrainerCallback

## GPU Detection and Configuration

def check_gpu_availability():
    """Check GPU availability and print information."""
    if torch.cuda.is_available():
        n_gpus = torch.cuda.device_count()
        print(f"Found {n_gpus} GPUs available!")

        for i in range(n_gpus):
            gpu_name = torch.cuda.get_device_name(i)
            gpu_memory = torch.cuda.get_device_properties(i).total_memory / 1e9  # Convert to GB
            print(f"GPU {i}: {gpu_name} with {gpu_memory:.2f} GB memory")

        return n_gpus
    else:
        print("No GPU available. Training will be slow on CPU.")
        return 0

# Check available GPUs
num_gpus = check_gpu_availability()


## Configuration

# Model Configuration
model_name = "gpt2-medium"  # Changed from gpt2-medium to reduce model size
output_dir = "./lyrics_generator_finetuned"  # Where fine-tuned model will be saved

# Dataset Configuration
dataset_path = "/kaggle/working/lyrics.txt"
val_split = 0.1  # Validation split ratio (0.1 = 10%)

# Training Configuration
block_size = 256  # Reduced from 512 to save memory
batch_size = 16  # Reduced from 16 to save memory
grad_accum = 8 # Increased from 8 to maintain effective batch size
epochs = 5  # Number of training epochs
learning_rate = 5e-5 # Peak learning rate

# Multi-GPU Configuration
use_distributed_training = num_gpus > 1  # Enable distributed training if multiple GPUs are available

# Optimization Configuration
use_peft = True  # Parameter-Efficient Fine-Tuning (PEFT)
lora_r = 16  # Reduced from 16
lora_alpha = 32  # Reduced from 32
lora_dropout = 0.05
use_4bit = False  # Disabled 4-bit quantization since bitsandbytes isn't available
fp16 = True  # Mixed precision training
gradient_checkpointing = True  # Save memory with gradient checkpointing



# Adjust batch size and gradient accumulation when using multiple GPUs
if num_gpus > 1:
    # We can increase batch size or reduce gradient accumulation with multiple GPUs
    # Aim for an effective batch size of batch_size * grad_accum * num_gpus
    effective_batch = batch_size * grad_accum
    print(f"\nInitial effective batch size: {effective_batch}")

    # Option 1: Keep same effective batch but increase per-device batch
    # Adjust batch_size based on available GPUs and memory (T4 has 16GB)
    # A per-device batch of 4-8 is usually feasible for gpt2-large with PEFT/FP16
    new_batch_size_per_device = min(8, batch_size * 2) # Example adjustment
    new_grad_accum = max(1, effective_batch // (new_batch_size_per_device * num_gpus)) # Adjust grad_accum

    batch_size = new_batch_size_per_device
    grad_accum = new_grad_accum

    print(f"\nAdjusted for {num_gpus} GPUs:")
    print(f"- Per-device train/eval batch size: {batch_size}")
    print(f"- Gradient accumulation steps: {grad_accum}")
    print(f"- Effective batch size: {batch_size * grad_accum * num_gpus}")

# Generation Configuration
prompt = "Love is like a"  # Prompt for text generation
max_new_tokens = 100  # Maximum number of new tokens to generate
temperature = 1.2  # Temperature for text generation (higher = more random)
top_k = 50  # Top-k sampling parameter
top_p = 0.95  # Top-p sampling parameter


## Data Loading and Preparation

def load_and_prepare_datasets(dataset_path, val_split=0.1):
    """Load, split and prepare datasets for training."""
    print(f"Loading dataset from {dataset_path}")

    try:
        # Load dataset from file
        # Assuming the file contains one example per line
        raw_datasets = load_dataset("text", data_files={"train": dataset_path})

        if 'train' not in raw_datasets or not raw_datasets['train']:
             raise ValueError(f"Training dataset at '{dataset_path}' is empty or could not be loaded correctly.")

        # Create validation split
        split_datasets = raw_datasets["train"].train_test_split(
            test_size=val_split, shuffle=True, seed=42
        )
        datasets = {"train": split_datasets["train"], "validation": split_datasets["test"]}

        # Display dataset information
        print(f"Dataset loaded with {len(datasets['train'])} training and {len(datasets['validation'])} validation examples")

        # Display sample from the dataset
        print("\nSample from the dataset:")
        for i in range(min(3, len(datasets['train']))):
            sample = datasets['train'][i]['text']
            print(f"{sample[:100]}..." if len(sample) > 100 else sample)

        return datasets

    except FileNotFoundError:
        print(f"Error: Dataset file not found at {dataset_path}")
        raise
    except Exception as e:
        print(f"Error loading dataset: {e}")
        raise


# Load the datasets
# Ensure the dataset file exists before running this
if not os.path.exists(dataset_path):
    print(f"Error: Dataset file not found at {dataset_path}. Please update dataset_path.")
    # Create a dummy file for demonstration if the file doesn't exist
    print("Creating a dummy lyrics.txt file for demonstration.")
    with open(dataset_path, "w") as f:
        f.write("This is a sample lyric.\n")
        f.write("Another line for the song.\n")
        f.write("And a third line to make it a bit longer.\n")
        f.write("Love is like a river, flowing free.\n")
        f.write("Love is like a mountain, strong and high.\n")
        f.write("Love is like a bird, soaring in the sky.\n")
        f.write("Music fills the air tonight.\n")
        f.write("Singing songs under the moonlight.\n")
        f.write("Dancing in the pale moonlight.\n")
    print(f"Dummy dataset created at {dataset_path}. Please replace with your actual data.")

datasets = load_and_prepare_datasets(dataset_path, val_split)


## Tokenizer Setup

def load_tokenizer(model_name):
    """Load and configure the tokenizer."""
    print(f"Loading tokenizer for model: {model_name}")

    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

        # Set padding token if it doesn't exist
        if tokenizer.pad_token is None:
            if tokenizer.eos_token is not None:
                tokenizer.pad_token = tokenizer.eos_token
                print(f"Set pad_token to eos_token ({tokenizer.eos_token})")
            else:
                # Add a new pad token if neither pad_token nor eos_token exists
                tokenizer.add_special_tokens({'pad_token': '[PAD]'})
                print("Added '[PAD]' as pad_token.")

        # Display tokenizer information
        print(f"Vocabulary size: {len(tokenizer)}")
        print(f"Model max length: {tokenizer.model_max_length}")
        print(f"Pad token ID: {tokenizer.pad_token_id}")

        return tokenizer

    except Exception as e:
        print(f"Error loading tokenizer '{model_name}': {e}")
        raise


# Load the tokenizer
tokenizer = load_tokenizer(model_name)


## Tokenize Dataset

def tokenize_dataset(datasets, tokenizer, block_size):
    """Tokenize and process the dataset."""
    print("Tokenizing dataset...")

    def tokenize_function(examples):
        """Tokenize text examples."""
        # Ensure padding=False here, padding will be handled by DataCollator
        return tokenizer(examples["text"], truncation=True, max_length=block_size, padding=False)

    # Apply tokenization to the dataset
    # Use a reasonable number of processes, capped by CPU cores
    num_proc = min(4, os.cpu_count() or 1) # Ensure at least 1 process
    print(f"Using {num_proc} processes for tokenization.")

    # Process both train and validation datasets separately
    tokenized_datasets = {}
    for split in ['train', 'validation']:
        # Check if the split exists and is not empty
        if split in datasets and len(datasets[split]) > 0:
            tokenized_datasets[split] = datasets[split].map(
                tokenize_function,
                batched=True,
                num_proc=num_proc,
                remove_columns=datasets[split].column_names,
                desc=f"Tokenizing {split} dataset"
            )
        else:
            print(f"Warning: '{split}' dataset split is empty or missing. Skipping tokenization for this split.")
            tokenized_datasets[split] = None # Explicitly set to None or an empty Dataset object

    # Filter out None values for splits that were empty
    tokenized_datasets = {k: v for k, v in tokenized_datasets.items() if v is not None}


    print(f"Tokenized dataset sizes: {len(tokenized_datasets.get('train', []))} train, {len(tokenized_datasets.get('validation', []))} validation")

    # Group texts into blocks for causal language modeling
    def group_texts(examples):
        """Group texts into blocks of maximum length."""
        # Concatenate all texts
        concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
        total_length = len(concatenated_examples[list(examples.keys())[0]])

        # Handle case where total_length < block_size
        if total_length == 0:
            # Return empty result if there's no text after tokenization
            return {k: [] for k in examples.keys()}

        if total_length < block_size:
            print(f"Warning: Total length ({total_length}) is less than block_size ({block_size}). Returning a single partial block.")
            result = {k: [t[0:total_length]] for k, t in concatenated_examples.items()}
            result["labels"] = result["input_ids"].copy()
            return result

        # Drop the small remainder for equal-sized chunks
        total_length = (total_length // block_size) * block_size

        # Split by chunks of block_size
        result = {
            k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
            for k, t in concatenated_examples.items()
        }

        # Create 'labels' - for Causal LM, labels are the input_ids shifted
        result["labels"] = result["input_ids"].copy()
        return result

    # Apply grouping function to both splits
    print("Grouping texts into blocks...")
    lm_datasets = {}
    for split in ['train', 'validation']:
         if split in tokenized_datasets and len(tokenized_datasets[split]) > 0:
            lm_datasets[split] = tokenized_datasets[split].map(
                group_texts,
                batched=True,
                num_proc=num_proc,
                desc=f"Grouping {split} dataset"
            )
            # Filter out empty examples that might result from grouping
            lm_datasets[split] = lm_datasets[split].filter(lambda example: len(example['input_ids']) > 0)
         else:
             print(f"Skipping grouping for empty or missing '{split}' dataset split.")
             lm_datasets[split] = None # Explicitly set to None

    # Filter out None values for splits that were empty after grouping
    lm_datasets = {k: v for k, v in lm_datasets.items() if v is not None and len(v) > 0}


    print(f"Final dataset sizes: {len(lm_datasets.get('train', []))} train, {len(lm_datasets.get('validation', []))} validation")

    # Show example text block
    if 'train' in lm_datasets and len(lm_datasets['train']) > 0:
        print("\nExample block of tokenized text:")
        example = lm_datasets['train'][0]
        print(f"Length: {len(example['input_ids'])}")
        decoded = tokenizer.decode(example['input_ids'][:50])
        print(f"Decoded sample: {decoded}...")

    return lm_datasets


# Tokenize and process the dataset
lm_datasets = tokenize_dataset(datasets, tokenizer, block_size)

# Check if datasets are empty after processing
if 'train' not in lm_datasets or len(lm_datasets['train']) == 0:
    print("\nError: Training dataset is empty after tokenization and grouping. Cannot proceed with training.")
    # Exit or handle this case appropriately, e.g., by skipping training
    training_possible = False
else:
    training_possible = True
    if 'validation' not in lm_datasets or len(lm_datasets['validation']) == 0:
        print("\nWarning: Validation dataset is empty after tokenization and grouping. Training will proceed without evaluation.")


## Model Loading

def load_model(model_name, tokenizer, use_4bit=False):
    """Load the pre-trained model with memory-optimized configurations."""
    print(f"Loading base model: {model_name}")

    # Quantization configuration
    bnb_config = None
    if use_4bit:
        # Skip this section since bitsandbytes isn't properly installed
        print("Warning: 4-bit quantization requested but bitsandbytes is not available.")
        print("Continuing without quantization. Install with: pip install -U bitsandbytes")

    try:
        # Load model with memory-optimized settings (without 4-bit quantization)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            # No quantization config
            device_map="auto" if num_gpus > 1 else "cuda:0",  # Explicit device mapping
            trust_remote_code=True,
            use_cache=False if gradient_checkpointing else True,
            torch_dtype=torch.float16 if fp16 else None,  # Explicitly set dtype
            low_cpu_mem_usage=True,  # Reduce CPU memory usage during loading
        )

        # Resize token embeddings if needed
        if len(tokenizer) > model.config.vocab_size:
            print(f"Resizing model token embeddings from {model.config.vocab_size} to {len(tokenizer)}")
            model.resize_token_embeddings(len(tokenizer))

        # Configure pad token ID
        if tokenizer.pad_token_id is not None:
            model.config.pad_token_id = tokenizer.pad_token_id
            print(f"Set model.config.pad_token_id to {tokenizer.pad_token_id}")

        # Print model information
        print(f"Model loaded: {model.config.model_type}, {model.num_parameters():,} parameters")
        print(f"Model device map: {model.hf_device_map if hasattr(model, 'hf_device_map') else 'cuda:0'}")

        # Enable memory efficient attention if available in model config (helps with memory usage)
        if hasattr(model.config, "use_memory_efficient_attention"):
            model.config.use_memory_efficient_attention = True
            print("Enabled memory efficient attention in model config")

        return model

    except Exception as e:
        print(f"Error loading model '{model_name}': {e}")
        raise




# Load the base model
# Only load the model if training is possible
if training_possible:
    model = load_model(model_name, tokenizer, use_4bit)
else:
    model = None # Set model to None if training is not possible


## PEFT/LoRA Application

def apply_peft(model, use_peft=True, lora_r=16, lora_alpha=32, lora_dropout=0.05,
               use_4bit=False, gradient_checkpointing=True):
    """Apply PEFT configuration to the model."""
    if not use_peft or model is None:
        print("Skipping PEFT/LoRA application")
        return model

    print("Applying PEFT (LoRA)...")

    # Handle quantized models
    if use_4bit:
        print("Preparing model for k-bit training...")
        # This function handles setting requires_grad=True for trainable parameters
        model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=gradient_checkpointing)
        print("Model prepared for k-bit training.")

        # Ensure gradient checkpointing is enabled on the model if requested
        if gradient_checkpointing and not model.is_gradient_checkpointing_enabled():
             print("Enabling gradient checkpointing on the model.")
             model.gradient_checkpointing_enable()


    # Map model architecture to appropriate target modules
    model_type = model.config.model_type if hasattr(model.config, "model_type") else "unknown"

    # Define target modules based on model type
    if model_type.startswith('gpt2'):
        # For GPT-2, common target modules include attention and MLP layers
        target_modules = ["c_attn", "c_proj", "c_fc"]
    elif model_type.startswith('llama') or model_type.startswith('mistral'):
        target_modules = ["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
    elif model_type.startswith('opt'):
        target_modules = ["q_proj", "v_proj", "k_proj", "out_proj", "fc1", "fc2"]
    elif model_type.startswith('bloom'):
         target_modules = ["query_key_value", "dense", "dense_h_to_4h", "dense_4h_to_h"]
    else:
        # Default target modules - may need adjustment for other models
        print(f"Unknown model type: {model_type}. Using default target modules. Please verify.")
        target_modules = ["query", "value", "key", "output", "dense"] # Common names


    print(f"Using target modules for LoRA: {target_modules}")

    # LoRA configuration
    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        r=lora_r,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout,
        target_modules=target_modules,
        bias="none", # Generally recommended to not train bias with LoRA
    )

    # Wrap the base model with PEFT
    model = get_peft_model(model, peft_config)

    # *** ADDED: Explicitly enable input gradients for gradient checkpointing with PEFT ***
    # This is often necessary when combining PEFT, quantization, and gradient checkpointing
    # to ensure that the inputs to the trainable layers have requires_grad=True.
    if gradient_checkpointing:
        print("Enabling input gradients for the model.")
        model.enable_input_require_grads()

    # Log trainable parameters info
    model.print_trainable_parameters()

    return model


# Apply PEFT if specified and model is loaded
if training_possible:
    model = apply_peft(model, use_peft, lora_r, lora_alpha, lora_dropout, use_4bit, gradient_checkpointing)


## Training Setup

def setup_training(model, tokenizer, lm_datasets, output_dir,
                   batch_size=2, grad_accum=16, learning_rate=2e-5, epochs=3,
                   fp16=True, gradient_checkpointing=True, use_distributed_training=False):
    """Set up model training with memory optimizations."""
    if model is None or 'train' not in lm_datasets or len(lm_datasets['train']) == 0:
        print("Skipping training setup: Model not loaded or training dataset is empty.")
        return None

    print("Setting up memory-optimized training...")

    # Define training arguments with memory optimizations
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        gradient_accumulation_steps=grad_accum,
        learning_rate=learning_rate,
        num_train_epochs=epochs,
        lr_scheduler_type="cosine",
        warmup_ratio=0.05,
        logging_dir=f"{output_dir}/logs",
        logging_strategy="steps",
        logging_steps=100,  # Log less frequently to reduce overhead
        save_strategy="epoch",
        eval_strategy="no",
        save_total_limit=1,  # Only keep the latest checkpoint to save disk space
        load_best_model_at_end=False,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        fp16=fp16,
        gradient_checkpointing=gradient_checkpointing,
        report_to="none",
        push_to_hub=False,
        
        # Memory optimization settings
        ddp_find_unused_parameters=False,
        local_rank=-1,
        dataloader_num_workers=0,  # Set to 0 to avoid extra memory usage
        remove_unused_columns=False,
        optim="adamw_torch",  # Use the PyTorch implementation of AdamW
        
        # These help with out-of-memory issues
        max_grad_norm=1.0,  # Clip gradients to prevent exploding gradients
        # eval_strategy="steps" if 'validation' in lm_datasets else "no",
        eval_steps=100 if 'validation' in lm_datasets else None,  # Evaluate less frequently
    )

    # Data collator with dynamic padding for memory efficiency
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=False
    )

    # Define simple metrics function (avoid complex calculations that might use extra memory)
    def compute_metrics(eval_pred):
        with torch.no_grad():
            logits, labels = eval_pred
            if isinstance(logits, tuple):
                logits = logits[0]
            loss = trainer.compute_loss(model, {"input_ids": labels, "attention_mask": None})
            try:
                perplexity = math.exp(loss)
                return {"perplexity": perplexity, "eval_loss": loss}
            except:
                return {"eval_loss": loss}

    # Setup trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=lm_datasets["train"],
        eval_dataset=lm_datasets.get("validation") if 'validation' in lm_datasets and len(lm_datasets["validation"]) > 0 else None,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics if 'validation' in lm_datasets and len(lm_datasets['validation']) > 0 else None,
    )

    return trainer


# Set up the trainer with multi-GPU support if available and training is possible
trainer = setup_training(
    model, tokenizer, lm_datasets, output_dir,
    batch_size, grad_accum, learning_rate, epochs,
    fp16, gradient_checkpointing, use_distributed_training
)

# Check if trainer was successfully set up
training_possible = trainer is not None


## Train the Model

def train_model(trainer, use_distributed_training=False):
    """Train the model and display progress."""
    if trainer is None:
        print("Skipping training: Trainer was not set up successfully.")
        return False

    print("Starting training...")

    try:
        if use_distributed_training:
            print("Using distributed training across multiple GPUs.")
            print("Ensure you launch this script using 'accelerate launch'.")

        # Start training
        # The Trainer handles multi-GPU distribution when launched correctly
        train_result = trainer.train()

        # Display training results
        print("\nTraining completed!")
        metrics = train_result.metrics
        print(f"Training metrics: {metrics}")

        # Save model and tokenizer
        print(f"Saving model to {trainer.args.output_dir}")
        # Save the PEFT model (adapter weights)
        trainer.model.save_pretrained(trainer.args.output_dir)
        # Save the tokenizer
        trainer.tokenizer.save_pretrained(trainer.args.output_dir)

        print("Model and tokenizer saved.")

        return True

    except Exception as e:
        print(f"Error during training: {e}")
        # Print more detailed traceback for debugging
        import traceback
        traceback.print_exc()
        return False


# If running in notebook with multiple GPUs, we need to use the accelerate launcher
# For simplicity, we'll include instructions for both notebook and script approaches

print("\n" + "="*80)
if use_distributed_training:
    print("MULTI-GPU TRAINING INSTRUCTIONS:")
    print("To run training with multiple GPUs, you should:")
    print("1. Save this notebook as a Python script (e.g., 'train_lyrics.py')")
    print("2. Install accelerate: pip install accelerate bitsandbytes peft transformers datasets evaluate")
    print("3. Configure accelerate: accelerate config")
    print("   - Run this in your terminal. It will ask you questions about your setup.")
    print("     * How many machines are you using? 1")
    print("     * Do you want to use Distributed Data Parallel? yes")
    print(f"     * How many GPUs you want to use on this machine? {num_gpus}")
    print(f"     * What is your compute type? {'fp16' if fp16 else 'no'}")
    print("     * Other options can be left as default unless you have specific needs.")
    print("4. Run the script using the accelerate launch command:")
    print("   accelerate launch train_lyrics.py")
    print("\nNote: Running this cell directly in a standard notebook environment")
    print("will likely only use one GPU unless accelerate is configured externally.")
    print("="*80 + "\n")
else:
     print("SINGLE-GPU OR CPU TRAINING:")
     print("Training will proceed on a single device.")
     print("="*80 + "\n")


# Train the model if setup was successful
training_successful = False
if training_possible:
    training_successful = train_model(trainer, use_distributed_training)
else:
    print("Skipping training due to setup issues (e.g., empty dataset).")


## Text Generation with the Fine-tuned Model


## How to use this script from command line for multi-GPU training
"""
To properly use this script with 2 T4 GPUs via command line:

1. Save this notebook as a Python script (e.g., 'train_lyrics.py')
2. Install accelerate: pip install accelerate bitsandbytes peft transformers datasets evaluate
3. Configure accelerate: accelerate config
   - Run this in your terminal. It will ask you questions about your setup.
   - Choose the following options:
     * How many machines are you using? 1
     * Do you want to use Distributed Data Parallel? yes
     * How many GPUs you want to use on this machine? 2 (or the number detected)
     * What is your compute type? Fp16 (if fp16=True in config) or No (if fp16=False)
     * Other options can be left as default unless you have specific needs.
4. Run the script using the accelerate launch command:
   accelerate launch train_lyrics.py

This will distribute the training process across both T4 GPUs using the configuration you set up with `accelerate config`.
"""


Found 2 GPUs available!
GPU 0: Tesla T4 with 15.83 GB memory
GPU 1: Tesla T4 with 15.83 GB memory

Initial effective batch size: 128

Adjusted for 2 GPUs:
- Per-device train/eval batch size: 8
- Gradient accumulation steps: 8
- Effective batch size: 128
Loading dataset from /kaggle/working/lyrics.txt
Dataset loaded with 72000 training and 8000 validation examples

Sample from the dataset:
well my time went too quickly i went lickety splitly out to my old fifty five as i pulled away slowl...
pull me in tonight and stay forever i wanna know the secrets of your heart let s begin tonight it s ...
now baby the day you were born they picked you up and wrapped you up so cold but so hot everything m...
Loading tokenizer for model: gpt2-medium
Set pad_token to eos_token (<|endoftext|>)
Vocabulary size: 50257
Model max length: 1024
Pad token ID: 50256
Tokenizing dataset...
Using 4 processes for tokenization.
Tokenized dataset sizes: 72000 train, 8000 validation
Grouping texts into blocks...
Fina

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(
/tmp/ipykernel_31/641374545.py:541: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.



MULTI-GPU TRAINING INSTRUCTIONS:
To run training with multiple GPUs, you should:
1. Save this notebook as a Python script (e.g., 'train_lyrics.py')
2. Install accelerate: pip install accelerate bitsandbytes peft transformers datasets evaluate
3. Configure accelerate: accelerate config
   - Run this in your terminal. It will ask you questions about your setup.
     * How many machines are you using? 1
     * Do you want to use Distributed Data Parallel? yes
     * How many GPUs you want to use on this machine? 2
     * What is your compute type? fp16
     * Other options can be left as default unless you have specific needs.
4. Run the script using the accelerate launch command:
   accelerate launch train_lyrics.py

Note: Running this cell directly in a standard notebook environment
will likely only use one GPU unless accelerate is configured externally.

Starting training...
Using distributed training across multiple GPUs.
Ensure you launch this script using 'accelerate launch'.


Step,Training Loss
100,3.808100
200,3.543300
300,3.397000
400,3.326400
500,3.314900
600,3.290100
700,3.271800
800,3.271200
900,3.264800
1000,3.258600


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.



Training completed!
Training metrics: {'train_runtime': 26929.83, 'train_samples_per_second': 11.698, 'train_steps_per_second': 0.183, 'total_flos': 1.4919306986913792e+17, 'train_loss': 3.25063321260902, 'epoch': 4.995429151853733}
Saving model to ./lyrics_generator_finetuned
Model and tokenizer saved.

Loading fine-tuned model from ./lyrics_generator_finetuned for generation...
Generating on device: cuda


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Generating text with prompt: 'Love is like a'


"\nTo properly use this script with 2 T4 GPUs via command line:\n\n1. Save this notebook as a Python script (e.g., 'train_lyrics.py')\n2. Install accelerate: pip install accelerate bitsandbytes peft transformers datasets evaluate\n3. Configure accelerate: accelerate config\n   - Run this in your terminal. It will ask you questions about your setup.\n   - Choose the following options:\n     * How many machines are you using? 1\n     * Do you want to use Distributed Data Parallel? yes\n     * How many GPUs you want to use on this machine? 2 (or the number detected)\n     * What is your compute type? Fp16 (if fp16=True in config) or No (if fp16=False)\n     * Other options can be left as default unless you have specific needs.\n4. Run the script using the accelerate launch command:\n   accelerate launch train_lyrics.py\n\nThis will distribute the training process across both T4 GPUs using the configuration you set up with `accelerate config`.\n"

In [22]:
# Generation Configuration
prompt = "Rain in Night"
max_new_tokens = 150
temperature = 1.1
top_k = 100
top_p = 0.9

def generate_text(model_path, tokenizer, prompt, max_new_tokens=100, temperature=0.8, top_k=50, top_p=0.9,repetition_penalty=1.3,no_repeat_ngram_size=3,do_sample=True,num_return_sequences=1):
    """Generate text using the fine-tuned model."""
    print(f"\nLoading fine-tuned model from {model_path} for generation...")

    try:
        # Determine the device for generation
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Generating on device: {device}")

        # Load the base model
        base_model = AutoModelForCausalLM.from_pretrained(
            model_name, # Use the original model name for the base model
            trust_remote_code=True,
            # Ensure use_cache is True for efficient generation
            use_cache=True,
        )

        # Load the PEFT adapter weights
        model = PeftModel.from_pretrained(base_model, model_path)

        # Move the model to the generation device
        model.to(device)
        model.eval() # Set model to evaluation mode

        # Prepare input - move to the generation device
        input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

        # Generate text
        print(f"Generating text with prompt: '{prompt}'")
        with torch.no_grad(): # No need to track gradients during generation
            output = model.generate(
                input_ids=input_ids,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_k=top_k,
                top_p=top_p,
                do_sample=True, # Enable sampling
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                num_return_sequences=1,
                repetition_penalty=repetition_penalty,          # <-- Add this
                no_repeat_ngram_size=no_repeat_ngram_size,
            )

        # Decode the generated text
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

        # Format and display results
        formatted_result = f"<h3>Generated Lyrics:</h3><pre>{generated_text}</pre>"
        display(HTML(formatted_result))

        return generated_text

    except Exception as e:
        print(f"Error during text generation: {e}")
        import traceback
        traceback.print_exc()
        return None

import re

# Generate text if training was successful
if training_successful:
    # Use the output directory where the fine-tuned model was saved
    generated_lyrics = generate_text(output_dir, tokenizer, prompt, max_new_tokens, temperature, top_k, top_p)
else:
    print("\nSkipping text generation as training was not successful.")
    print("Please ensure the dataset path is correct and training completed without errors.")

lines = re.split(r'(?<=[.!?])\s+|\n', generated_lyrics.strip())
formatted = "\n".join(line.strip() for line in lines if line.strip())
print(formatted)



Loading fine-tuned model from ./lyrics_generator_finetuned for generation...
Generating on device: cuda
Generating text with prompt: 'Rain in Night'


Rain in Night came out of nowhere the fog was thick this morning no one moved by they were stuck on that same road trying to find a highway i want them all and if you can lift up some rocks please don t break their necks they are standing right there beside us looking down at me we ll make it together when day ends these two sides just might meet again tonight tell my people take every penny come your way oh yeah now then go around show em what s yours do not let her put things past with so little time she could leave for someone else but well before each sunrise something like always comes along hey whoo ho look back over here is somebody gettin married or getting divorced somethin new gotta be done gonna see how yuh did holdin onto


In [13]:
!pip install git+https://github.com/suno-ai/bark.git

  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-v55u703e
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-v55u703e
  Resolved https://github.com/suno-ai/bark.git to commit f4f32d4cd480dfec1c245d258174bc9bde3c2148
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 33.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for suno-bark: filename=suno_bark-0.0.1a0-py3-none-any.whl size=2567395 sha256=ec254f591eaf8fbdbe8b720633b9075fbda9a11286d1d4f4879af48164e134a1
  Stored in directory: /tmp/pip-ephem-wheel-cache-9jqj0epn/wheels/d7/12/5e/2c493bffde263f84a51c141ad68c113365714a1e0ef0051c9b
  Created wheel for encodec: filename=encodec-0.1.1-py3-none-any.whl size=45760 sha256=bf5eb49dadce4783870232fe9498eae5588cf0030a37ad9155bd0a403b65d596
  S

In [23]:
from transformers import AutoProcessor, BarkModel

processor = AutoProcessor.from_pretrained("suno/bark")
model = BarkModel.from_pretrained("suno/bark")

voice_preset = "v2/en_speaker_6"

inputs = processor("Love is like a big fish in a sea of slime that you can hear the waves, you can hear the waves of the ocean, and you can hear the waves of the ocean that is drowning out the sea that you can hear the waves that the sea is drowning out the sea that you can hear the waves that the sea is drowning out the sea that you can hear the waves that the sea is drowning out the sea that you can hear the waves that the sea is drowning out the sea that you can hear the waves", voice_preset=voice_preset)

audio_array = model.generate(**inputs)
audio_array = audio_array.cpu().numpy().squeeze()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [24]:
from IPython.display import Audio

sample_rate = model.generation_config.sample_rate
Audio(audio_array, rate=sample_rate)

In [25]:
import scipy

sample_rate = model.generation_config.sample_rate
scipy.io.wavfile.write("bark_out.wav", rate=sample_rate, data=audio_array)

In [26]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [27]:
inputs = processor("♪ Love is like a big fish in a sea of slime that you can hear the waves, you can hear the waves of the ocean, and you can hear the waves of the ocean that is drowning out the sea that you can hear the waves that the sea is drowning out the sea that you can hear the waves that the sea is drowning out the sea that you can hear the waves that the sea is drowning out the sea that you can hear the waves that the sea is drowning out the sea that you can hear the waves ♪")

# generate speech
speech_output = model.generate(**inputs.to(device))

Audio(speech_output[0].cpu().numpy(), rate=sample_rate)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [23]:
import zipfile
import os

def zip_folder(folder_path, output_zip_path):
    with zipfile.ZipFile(output_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Walk the directory
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Create the full file path
                file_path = os.path.join(root, file)
                # Write the file with arcname to preserve folder structure
                arcname = os.path.relpath(file_path, folder_path)
                zipf.write(file_path, arcname)

# Example usage
zip_folder('/kaggle/working/lyrics_generator_finetuned', 'my_folder.zip')
